In [20]:
from hypersurface_tf import *
from generate_h import *
import sympy as sp
from pprint import pprint
from scipy.optimize import minimize
from sympy.utilities.iterables import flatten
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf

In [21]:
z0, z1, z2, z3, z4= sp.symbols('z0, z1, z2, z3, z4')
Z = [z0,z1,z2,z3,z4]
f = z0**5 + z1**5 + z2**5 + z3**5 + z4**5 + 0.5*z0*z1*z2*z3*z4
HS = Hypersurface(Z, f, 10000)

In [22]:
k = 6
HS.set_k(k)

In [23]:
factor = HS.integrate(lambda patch: patch.num_eta_tf('FS'), holomorphic=True, tensor=True)

In [24]:
factor = factor.numpy()

Normalization factor of eta

In [25]:
h_sym = get_sym_info(HS)

In [26]:
def train(HS, g, learning_rate, factor):
    with tf.GradientTape() as t:
        h = tf.matmul(g, g, adjoint_b=True)
        h = h/h[0][0]
        integration = HS.integrate(lambda patch: tf.abs(patch.num_eta_tf(h)/factor - 1), tensor=True)
    dg = t.gradient(integration, g)
    g.assign_sub(learning_rate * dg)

In [27]:
g0 = np.linalg.cholesky(param_to_matrix(initial_FS_param(HS, h_sym), h_sym))
g0 = np.array(g0, dtype=np.complex64)
with tf.device('/cpu:0'):
    g = tf.Variable(g0)

In [30]:
learning_rate = 1

In [32]:
epochs = range(300)
for epoch in epochs:
    train(HS, g, learning_rate, factor)

    if epoch % 20 == 0:
        h = tf.matmul(g, g, adjoint_b=True)
        h = h/h[0][0]
        integration = HS.integrate(lambda patch: tf.abs(patch.num_eta_tf(h)/factor - 1), tensor=True)
        print(integration)

tf.Tensor(0.26734912, shape=(), dtype=float32)
tf.Tensor(0.26179087, shape=(), dtype=float32)
tf.Tensor(0.25621593, shape=(), dtype=float32)
tf.Tensor(0.25061625, shape=(), dtype=float32)
tf.Tensor(0.2449829, shape=(), dtype=float32)
tf.Tensor(0.23932876, shape=(), dtype=float32)
tf.Tensor(0.23366109, shape=(), dtype=float32)
tf.Tensor(0.22797656, shape=(), dtype=float32)
tf.Tensor(0.22228624, shape=(), dtype=float32)
tf.Tensor(0.21659584, shape=(), dtype=float32)
tf.Tensor(0.21092415, shape=(), dtype=float32)
tf.Tensor(0.2052735, shape=(), dtype=float32)
tf.Tensor(0.19965355, shape=(), dtype=float32)
tf.Tensor(0.1940877, shape=(), dtype=float32)
tf.Tensor(0.18858223, shape=(), dtype=float32)


In [ ]:
h_t = tf.matmul(g, g, adjoint_b=True)
h_t = h_t/h_t[0][0]
print(h_t[-1])

In [ ]:
param_3 = [-0.15478935, -0.07456169]
param_4 = [0.17843688, 0.69614476, 0.13144945, 0.75676199, 0.7845432 , 0.00997926]
g0 = initial_param_from_lowerk(HS, h_sym, param_3)

In [15]:
h_minimal = param_to_matrix([0.17843688, 0.69614476, 0.13144945, 0.75676199, 0.7845432 , 0.00997926], h_sym)

In [23]:
h_minimal[-1]

array([0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.26413405-0.00263595j, 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0.        +0.j        , 0.        +0.j        ,
       0. 

In [ ]:
sigma = HS.integrate(lambda patch: tf.abs(patch.num_eta_tf(h_minimal)/factor - 1), tensor=True)
delta_sigma = math.sqrt(HS.integrate(lambda patch: (tf.abs(patch.num_eta_tf(h_minimal)/factor - 1) - sigma)**2, 
                                     tensor=True) / HS.n_points)

In [ ]:
print(sigma)
print(delta_sigma)